In [62]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [63]:
dbName = "Crunchbase_Dataset"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
db = client.get_database()

mongodb://localhost/Crunchbase_Dataset


``lo primero que voy a hacer es filtrar las empresas con category_web``

In [64]:
curse = db.companies.find({'category_code':{ '$eq': 'games_video'}},{'name':1,'_id': 0, 'offices':1, 'total_money_raised':1,})
df = pd.DataFrame(curse)
df

,name,total_money_raised,offices
0,Flektor,$0,"[{'description': None, 'address1': '8536 Natio..."
1,Lala,$44.2M,"[{'description': 'Lala Headquarters', 'address..."
2,Joost,$45M,"[{'description': '', 'address1': '100 5th Ave ..."
3,Babelgum,$13.2M,"[{'description': '', 'address1': '', 'address2..."
4,Sparter,$0,"[{'description': None, 'address1': None, 'addr..."
...,...,...,...
1078,Symposium Productions,$0,[]
1079,Rev Media,$0,"[{'description': 'HQ', 'address1': '', 'addres..."
1080,Fictionaut,$0,"[{'description': 'New York', 'address1': '32-1..."
1081,Tiny Speck,$17.2M,"[{'description': '', 'address1': '163 Hastings..."


In [65]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
1,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
2,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
3,,,,,London,None,GBR,53.344104,-6.267494
4,None,None,None,None,None,None,USA,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...
1079,HQ,,,,Dallas,TX,USA,NaN,NaN
1080,New York,32-14 42nd St,,11103,Astoria,NY,USA,40.759795,-73.916127
1080,Houston,2208 Colquitt,,77098,Houston,TX,USA,29.735462,-95.413243
1081,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217


``Vemos cuales son los paises que más compañias gamers tienen``

In [66]:
offices.country_code.value_counts().head(5)

USA    587
GBR     74
CAN     35
DEU     30
FRA     26
Name: country_code, dtype: int64

In [67]:
df_concat  = pd.concat([df, offices], axis=1)
df_concat.drop(columns= 'offices')

,name,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Flektor,$0,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
1,Lala,$44.2M,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
2,Joost,$45M,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
3,Babelgum,$13.2M,,,,,London,None,GBR,53.344104,-6.267494
4,Sparter,$0,None,None,None,None,None,None,USA,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...,...
1079,Rev Media,$0,HQ,,,,Dallas,TX,USA,NaN,NaN
1080,Fictionaut,$0,New York,32-14 42nd St,,11103,Astoria,NY,USA,40.759795,-73.916127
1080,Fictionaut,$0,Houston,2208 Colquitt,,77098,Houston,TX,USA,29.735462,-95.413243
1081,Tiny Speck,$17.2M,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217


In [68]:
df_concat[df_concat['country_code'] == "USA"] 

,name,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Flektor,$0,"[{'description': None, 'address1': '8536 Natio...",None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
1,Lala,$44.2M,"[{'description': 'Lala Headquarters', 'address...",Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
2,Joost,$45M,"[{'description': '', 'address1': '100 5th Ave ...",,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
4,Sparter,$0,"[{'description': None, 'address1': None, 'addr...",None,None,None,None,None,None,USA,37.090240,-95.712891
5,Kyte,$23.4M,"[{'description': None, 'address1': '442 Post S...",None,442 Post Street,10th Floor,94102,San Francisco,CA,USA,37.788482,-122.409173
...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Shape Games,$0,"[{'description': '', 'address1': '901 North Pi...",,901 North Pitt str.,Suite 325,2231420,Alexandria,VA,USA,38.814208,-77.042423
1074,G5 Entertainment,$0,"[{'description': 'Development Office', 'addres...",US Marketing Office,"Suite 136, 1750 Montgomery Street",,94111,San Francisco,CA,USA,NaN,NaN
1079,Rev Media,$0,"[{'description': 'HQ', 'address1': '', 'addres...",HQ,,,,Dallas,TX,USA,NaN,NaN
1080,Fictionaut,$0,"[{'description': 'New York', 'address1': '32-1...",New York,32-14 42nd St,,11103,Astoria,NY,USA,40.759795,-73.916127


``Vemos dentro de USA cual es el estado que más empresas de gamers tienen y nos acercamos con el zip_code``

In [69]:
df_concat.state_code.value_counts().head()

CA    279
NY     88
WA     27
TX     23
MA     21
Name: state_code, dtype: int64

In [70]:
df_concat.zip_code.value_counts().head()

         181
94107     17
94105     14
94301      9
10013      8
Name: zip_code, dtype: int64

In [71]:
filtered = df_concat[(df_concat['state_code'] == "CA") & ((df_concat['zip_code'] == "94107")|(df_concat['zip_code'] == "94105"))]
filtered

,name,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
11,Ustream,$60.1M,"[{'description': 'San Francisco Office', 'addr...",San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480
14,Revision3,$9M,"[{'description': 'HQ', 'address1': '2415 3rd S...",HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243
15,CastTV,$3.1M,"[{'description': None, 'address1': '374 Branna...",None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913
21,blinkx,$0,"[{'description': None, 'address1': '1 Market S...",None,1 Market Street,"19th Floor, Spear Tower",94105,San Francisco,CA,USA,37.793819,-122.395089
24,hi5,$52M,"[{'description': '', 'address1': '55 Second St...",,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558
43,Metacafe,$50M,"[{'description': '', 'address1': '128 King Str...",,128 King Street,3rd Floor,94107,San Francisco,CA,USA,37.437328,-122.159928
45,IODA,$0,"[{'description': None, 'address1': '539 Bryant...",None,"539 Bryant Street, Suite 303",None,94107,San Francisco,CA,USA,37.780134,-122.396744
46,Kongregate,$19M,"[{'description': '', 'address1': '2 Shaw Alley...",,2 Shaw Alley,4th Floor,94105,San Francisco,CA,USA,37.786942,-122.401245
52,Mevio,$48.9M,"[{'description': '', 'address1': '577 2nd St S...",,577 2nd St Suite 203,,94107,San Francisco,CA,USA,37.782263,-122.392142
66,Crunchyroll,$4.8M,"[{'description': '', 'address1': '', 'address2...",,,,94105,San Francisco,CA,USA,37.781265,-122.393229


In [72]:
filtered.shape

(31, 12)

In [73]:
filtered_companies = filtered.drop(columns = 'offices')

In [74]:
import numpy as np

def transformToGeoPoint(filtered_companies):
    if np.isnan(filtered_companies.latitude) or np.isnan(filtered_companies.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[filtered_companies.longitude, filtered_companies.latitude]
    }
    



In [75]:
filtered_companies["location"] = filtered_companies.apply(transformToGeoPoint, axis=1)

In [76]:
filtered_companies.head()

,name,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
11,Ustream,$60.1M,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ..."
14,Revision3,$9M,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,"{'type': 'Point', 'coordinates': [-122.388243,..."
15,CastTV,$3.1M,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
21,blinkx,$0,None,1 Market Street,"19th Floor, Spear Tower",94105,San Francisco,CA,USA,37.793819,-122.395089,"{'type': 'Point', 'coordinates': [-122.395089,..."
24,hi5,$52M,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."


In [77]:
filtered_companies.to_json("output/gaming_companies.json",orient="records")